In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, MetaData
from sqlalchemy import Column, Integer, String, Numeric, Text, Float
import os

#Checking curent directory
os.getcwd()

#Setting curent directory
os.chdir('C:/Work/UCB-DS/Class Notes/Assignment/11')

In [2]:
#Use the engine and connection string to create a database called hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

#Create a connection to the engine called `conx`
conx = engine.connect()

#Use declarative_base and create ORM classes for each table.
Base = declarative_base()

In [3]:
#Create Measurement Class
class Measurements(Base):
    __tablename__ = 'measurements'
    id = Column(Integer, primary_key=True)
    station = Column(String(20))
    date = Column(String(20))
    prcp = Column(Float)
    tobs = Column(Integer)

    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    

In [4]:
#Create Station class
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(String(20))
    name = Column(String(45))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [5]:
#Create the tables in the database using create_all
Base.metadata.create_all(engine)

#Loading previously saved clean csv files into pandas dataframes
measurements_df = pd.read_csv('clean_hawaii_measurements.csv')
stations_df = pd.read_csv('clean_hawaii_stns.csv')

In [6]:
#Use Orient='records' to create a list of data to write
wtherData = measurements_df.to_dict(orient='records')
stnsData = stations_df.to_dict(orient='records')
wtherData[0]

#Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()    

In [7]:
#Save the references in table
wtherTable = sqlalchemy.Table('measurements', metadata, autoload=True)
stnsTable = sqlalchemy.Table('station', metadata, autoload=True)        

# Use `table.delete()` to remove any existing data
conx.execute(wtherTable.delete())
conx.execute(stnsTable.delete())

In [8]:
#Insert the data into the table
conx.execute(wtherTable.insert(), wtherData)
conx.execute(stnsTable.insert(), stnsData)

#Select records for first 10 rows
conx.execute("select * from measurements limit 10").fetchall()
conx.execute("select * from station").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (6, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 (7, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9),
 (8, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 (9, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]